In [2]:
from collections import defaultdict
import jellyfish

In [3]:
import spacy

In [4]:
rq1_df = pd.read_csv('processed01_par10-20.csv',index_col=0)

In [5]:
rq1_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 341805 entries, 0 to 341804
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Unnamed: 0        341805 non-null  int64 
 1   member_name       341805 non-null  object
 2   sitting_date      341805 non-null  object
 3   political_party   341805 non-null  object
 4   government        341805 non-null  object
 5   roles             341805 non-null  object
 6   member_gender     341805 non-null  object
 7   speech            341805 non-null  object
 8   year              341805 non-null  int64 
 9   is_government     341805 non-null  int64 
 10  speaker_gov_role  60473 non-null   object
 11  leadership_role   10689 non-null   object
 12  speech_clean      338501 non-null  object
dtypes: int64(3), object(10)
memory usage: 36.5+ MB


In [59]:
nlp = spacy.load("el_core_news_sm")

def tokenize_and_lemmatize(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_space]

In [76]:
# Load lexicon into dictionary
lexicon_df = pd.read_csv("out_lexicon_6sent.csv")

In [77]:
lexicon_df.head()

term  anger  disgust  fear  happiness  sadness  surprise
0   αβαφτιστος    4.00     4.50  1.00        1.0     2.50      4.50
1      Χριστος    4.50     3.75  4.25        4.0     4.00      4.50
2            α    3.75     4.00  4.00        4.0     3.75      4.75
3   αβαπτιστος    4.00     4.50  1.00        1.0     2.50      4.50
4  αβεβαιοτητα    1.00     1.00  2.50        1.0     1.50      1.00

In [78]:
lexicon = {
    row['term'].strip(): [
        float(row['anger']),
        float(row['disgust']),
        float(row['fear']),
        float(row['happiness']),
        float(row['sadness']),
        float(row['surprise'])
    ]
    for _, row in lexicon_df.iterrows()
}

In [29]:
def sent6_vec(text, lexicon, tokenize_fn):
    word_vecs = []

    for word in tokenize_fn(text):
        if word in lexicon:
            word_vecs.append(lexicon[word])

    if not word_vecs:
        return [0] * 6  
    
    word_vecs = np.array(word_vecs)

    rms = np.sqrt(np.mean(np.square(word_vecs), axis=0))
    return [round(v, 3) for v in rms]

<b style="color:blue;">Test to Sample - Started</b>

In [80]:
sample_df = rq1_df.sample(n=1000, random_state=32).copy()

In [79]:
print("αγαπημενη" in lexicon)
print(lexicon.get("αγαπημενη"))

True
[1.0, 1.0, 1.0, 4.75, 1.0, 2.75]


In [44]:
sample_df['speech_clean'] = sample_df['speech_clean'].fillna('').astype(str)

In [82]:
sample_df['sent6_vec'] = sample_df['speech_clean'].progress_apply(
    lambda text: sent6_vec(text, lexicon, tokenize_and_lemmatize)
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [83]:
sample_df[['member_name', 'political_party', 'year', 'roles', 'is_government', 'sent6_vec']].head()


member_name  \
318719  αναγνωστοπουλου πετρου αθανασια (σια)   
126402                 κωνσταντοπουλου ν. ζωη   
321719                μελας παναγιωτη ιωαννης   
257082            βαρδακης δημητριου σωκρατης   
65452            χρυσοχοιδης βασιλειου μιχαηλ   

                            political_party  year  \
318719  συνασπισμος ριζοσπαστικης αριστερας  2019   
126402  συνασπισμος ριζοσπαστικης αριστερας  2014   
321719                       νεα δημοκρατια  2019   
257082  συνασπισμος ριζοσπαστικης αριστερας  2017   
65452        πανελληνιο σοσιαλιστικο κινημα  2012   

                                                    roles  is_government  \
318719  ['αναπληρωτης υπουργος εξωτερικων(15/02/2019-0...              0   
126402                                      ['βουλευτης']              0   
321719                                      ['βουλευτης']              0   
257082                                      ['βουλευτης']              1   
65452   ['υπουργος αναπτυξης ανταγωνιστικοτητας και να...              1   

                                         sent6_vec  
318719    [3.142, 2.915, 2.016, 1.581, 1.0, 3.142]  
126402                          [0, 0, 0, 0, 0, 0]  
321719                          [0, 0, 0, 0, 0, 0]  
257082     [2.63, 2.517, 1.744, 2.457, 1.0, 3.131]  
65452   [3.651, 3.391, 2.363, 1.969, 1.225, 3.582]

In [84]:
sentiment_cols = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']
sentiment_df = pd.DataFrame(sample_df['sent6_vec'].tolist(), columns=sentiment_cols)

In [85]:
sample_df_sent = pd.concat([sample_df, sentiment_df], axis=1)

In [86]:
sample_df_sent[sentiment_cols].describe()

anger      disgust         fear    happiness      sadness  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000   
mean      1.769585     1.588783     1.115906     1.067624     0.714237   
std       1.644246     1.505850     1.040254     0.995051     0.668034   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       2.000000     1.500000     1.000000     1.000000     1.000000   
75%       3.261000     3.026000     2.109000     1.969000     1.145000   
max       4.330000     4.000000     4.250000     4.500000     3.500000   

          surprise  
count  1000.000000  
mean      1.915119  
std       1.727340  
min       0.000000  
25%       0.000000  
50%       2.500000  
75%       3.500000  
max       4.330000

<b style="color:blue;">Test to Sample - END</b>

In [90]:
rq1_df['sent6_vec'] = rq1_df['speech_clean'].progress_apply(
    lambda text: sent6_vec(text, lexicon, tokenize_and_lemmatize)
)

  0%|          | 0/341805 [00:00<?, ?it/s]

In [91]:
sentiment_cols = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']
sentiment_redf = pd.DataFrame(rq1_df['sent6_vec'].tolist(), columns=sentiment_cols)

In [92]:
meta_cols = ['member_name', 'political_party', 'year', 'is_government', 'roles']
meta_df = rq1_df[meta_cols].reset_index(drop=True)

In [93]:
rq1_results_df = pd.concat([meta_df, sentiment_redf], axis=1)

<b style="color:red;">Checkpoint-Saved</b>

In [94]:
rq1_results_df.to_csv('RQ1_results_df.csv')

In [99]:
rq1_results_df.head()

member_name political_party  year  is_government  \
0  τσιαρας αλεξανδρου κωνσταντινος  νεα δημοκρατια  2010              0   
1        ζωης κωνσταντινου χρηστος  νεα δημοκρατια  2010              0   
2        ζωης κωνσταντινου χρηστος  νεα δημοκρατια  2010              0   
3   ταλιαδουρος αθανασιου σπυριδων  νεα δημοκρατια  2010              0   
4     χαρακοπουλος παντελη μαξιμος  νεα δημοκρατια  2010              0   

           roles  anger  disgust   fear  happiness  sadness  surprise  
0  ['βουλευτης']  3.394    3.118  2.192      2.021    1.084     3.512  
1  ['βουλευτης']  3.275    3.100  2.085      1.740    1.067     3.413  
2  ['βουλευτης']  3.913    3.522  2.366      1.803    1.173     3.965  
3  ['βουλευτης']  3.386    3.188  2.176      2.101    1.101     3.674  
4  ['βουλευτης']  2.779    2.680  2.080      1.857    1.280     3.037

## 📒02 | RQ2 SMOG Index Greek Analysis based on Dritsa (2018)

**Preprocessing steps specifically tailored for this analysis:**\
✅ diacritics removal\
✅ formal phrases and honorifics\
✅ remove extra white space

In [6]:
rq2_df = pd.read_csv('processed01_par10-20.csv',index_col=0)

In [7]:
rq2_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 341805 entries, 0 to 341804
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Unnamed: 0        341805 non-null  int64 
 1   member_name       341805 non-null  object
 2   sitting_date      341805 non-null  object
 3   political_party   341805 non-null  object
 4   government        341805 non-null  object
 5   roles             341805 non-null  object
 6   member_gender     341805 non-null  object
 7   speech            341805 non-null  object
 8   year              341805 non-null  int64 
 9   is_government     341805 non-null  int64 
 10  speaker_gov_role  60473 non-null   object
 11  leadership_role   10689 non-null   object
 12  speech_clean      338501 non-null  object
dtypes: int64(3), object(10)
memory usage: 36.5+ MB


In [9]:
rq2_df['speech_clean'][100]

'με συγχωρειτε πολυ η ελληνικη κυβερνηση ειναι αυτη η οποια το απεσυρε απο τα προγραμματα χρηματοδοτησεως της ευρωπαικης ενωσης για ενα και μονο λογο για να μην αλλαξει ο ρολος και ο στοχος του εργου γιατι ακριβως αν αλλαζε ο στοχος του εργου τοτε θα επρεπε να καλυφθει απο τις οδηγιες οι οποιες ηταν μεταγενεστερες'

In [10]:
rq2_df = rq2_df.drop(columns=['Unnamed: 0', 'speech_clean'])

Text preprocessing function designed specifically for SMOG index readability analysis in Greek. 
- Diacritics are removed and all text is normalized to lowercase.  
- Sentence-ending punctuation (e.g., question marks and exclamation points) is standardized to periods to facilitate sentence boundary detection.  
- Formal and honorific expressions common in parliamentary speech are filtered using regular expressions.  
- Numeric digits are converted into their Greek textual form using the *invoice* library to reflect actual syllabic content.
- Only Greek letters, periods, and spaces are retained; all other symbols are removed.  
- Excess whitespace is collapsed to ensure a clean final output suitable for syllable and sentence segmentation.

In [10]:
def prepare_text_for_smog(text):
    if not isinstance(text, str):
        return None

    #Remove Greek diacritics
    text = remove_greek_diacritics(text)

    #Normalize to lowercase
    text = text.lower()

    #Keep sentence punctuation: periods (.), question marks (?), exclamation (!)
    #Replace them all with periods for SMOG sentence splitting
    text = text.replace('!', '.').replace('?', '.')

    #Remove formal/honorific expressions
    patterns_to_remove = [
        r'κυριε\s+(υπουργε|υφυπουργε|προεδρε|αντιπροεδρε)',
        r'\bκ\.\s*', 
        r'\bευχαριστω\b',
        r'\bαγαπητοι συναδελφοι\b',
        r'\bκυριοι συναδελφοι\b',
        r'\bκυριες\b'
    ]
    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text)

    #Convert digits to text using invoice library (e.g., "2023" → "δυο χιλιαδες είκοσι τρία")
    def normalize_numbers(match):
        try:
            return invoice.num_to_text(int(match.group()))
        except:
            return match.group()

    text = re.sub(r'\d+', normalize_numbers, text)

    #Remove unwanted symbols (keep Greek letters, spaces, periods)
    text = re.sub(r'[^α-ωΑ-Ωά-ώΆ-Ώ0-9. ]+', ' ', text)

    #Collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [13]:
rq2_df['speech_for_smog'] = rq2_df['speech'].progress_apply(prepare_text_for_smog)

  0%|          | 0/341805 [00:00<?, ?it/s]

In [4]:
!pip install greek-accentuation invoice nltk

In [5]:
import math
import spacy
from greek_accentuation.syllabify import syllabify
import invoice

In [6]:
nlp = spacy.load("el_core_news_sm")

The linguistic complexity of a Greek parliamentary speech was estimated using a localized adaptation of the SMOG index, as proposed by Dritsa (2018). Speeches with fewer than 60 sentences are excluded to align with Dritsa’s methodology, which requires a 30-sentence sample for reliable estimation.

- To ensure representativeness, 10 centered sentences are extracted from each third of the speech.  
- The number of polysyllabic words is then calculated using the `syllabify()` function from the `greek_accentuation` package.  

The SMOG score is computed using the standard formula:\
$SMOG = 1.0430 × √(polysyllables × 30 / sentences) + 3.1291$,  

interpreted **comparatively** (not as literal schooling years), due to limitations in applying English-based readability indices to Greek. This approach prioritizes consistency with existing Greek-language studies, enabling comparison across government and opposition rhetoric.

In [7]:
def compute_smog(text):
    if not isinstance(text, str):
        return None

    #Greek sentence tokenizer
    sentences = [sent.text for sent in nlp(text).sents]

    #SMOG needs at least 60 sentences to sample 30 | Dritsa 2018 followed
    if len(sentences) < 60:
        return None

    #get 20 middle sentences from a chunk
    def middle_20(sent_list):
        center = len(sent_list) // 2
        return sent_list[center - 10 : center + 10]

    #sample 10 from each third of the speech
    part1 = middle_20(sentences[:len(sentences)//3])
    part2 = middle_20(sentences[len(sentences)//3:2*len(sentences)//3])
    part3 = middle_20(sentences[2*len(sentences)//3:])
    sample_sentences = part1 + part2 + part3

    polysyllables = 0

    for sentence in sample_sentences:
        words = sentence.split()  # token-level accuracy is not critical for SMOG
        for word in words:
            if word.isalpha():
                try:
                    if len(syllabify(word)) > 2:
                        polysyllables += 1
                except:
                    continue

    # SMOG formula (based on 30-sentence sample)
    smog = 1.0430 * math.sqrt((polysyllables * 30) / len(sample_sentences)) + 3.1291
    return round(smog, 2)

<b style="color:blue;">Test to Sample</b>

In [15]:
sample_df = rq2_df[rq2_df['speech'].notna()].sample(1000, random_state=42).copy()

In [16]:
sample_df['smog_score'] = sample_df['speech_for_smog'].apply(compute_smog)

In [17]:
sample_df['smog_score'].describe()

count    19.000000
mean     17.904737
std       1.535442
min      14.120000
25%      17.110000
50%      17.470000
75%      19.215000
max      20.570000
Name: smog_score, dtype: float64

<b style="color:blue;">Test to Sample</b>

In [18]:
rq2_df['smog_score'] = rq2_df['speech_for_smog'].progress_apply(compute_smog)

  0%|          | 0/341805 [00:00<?, ?it/s]

In [19]:
columns_to_keep = [
    'member_name', 'year', 'political_party', 'is_government',
    'speaker_gov_role', 'leadership_role', 'roles', 'smog_score'
]
rq2_results_df = rq2_df[columns_to_keep].copy()